# DataThon ONU - Generation/treatment of datasets

This notebook aims to generate organized datasets for the RJDados group in the context of DataThon ONU 2023. All datasets are related to the group's theme, which aims to propose solutions to food insecurity in the state of Rio de Janeiro using public data.

Four datasets will be used:

- <b>CadÚnico</b>: Number of people in CadÚnico according to income range, by municipality.
- <b>SISVAN</b>: Number of people consuming three daily meals, by municipality.
- <b>SISVAN</b>: Number of people in different BMI (Body Mass Index) ranges, by municipality.
- <b>IBGE</b>: Listing of codes for Brazilian municipalities.
- <b>IBGE</b>: Population in each Brazilian municipality.

We start by importing the necessary libraries.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import glob
import geopandas as gpd
import folium
from folium import Choropleth, GeoJson
import plotly.express as px
from ipywidgets import interact, widgets
from scipy.spatial.distance import cdist
from math import radians

Now, we load the datasets used in the analysis. Note that different formats are used due to the various sources of the files.

We start by importing the municipality codes from IBGE.

In [2]:
codigos_ibge = pd.read_excel('./Data/CODIGOS_IBGE_MUNICIPIOS.xls', skiprows=range(6))

codigos_ibge_rj = codigos_ibge[codigos_ibge.Nome_UF=="Rio de Janeiro"][["Nome_Município",
                                                                        "Código Município Completo"]]
codigos_ibge_rj.columns = ['nome_mun', 'cod_mun_completo']

codigos_ibge_rj['cod_mun_ibge'] = codigos_ibge_rj['cod_mun_completo'].astype(str).str[:6]

codigos_ibge_rj.to_csv('./Data/Processed_Data/codigos_ibge_rj.csv', index=False)

codigos_ibge_rj

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


,nome_mun,cod_mun_completo,cod_mun_ibge
3175,Angra dos Reis,3300100,330010
3176,Aperibé,3300159,330015
3177,Araruama,3300209,330020
3178,Areal,3300225,330022
3179,Armação dos Búzios,3300233,330023
...,...,...,...
3262,Três Rios,3306008,330600
3263,Valença,3306107,330610
3264,Varre-Sai,3306156,330615
3265,Vassouras,3306206,330620


# Generation of dataset on CadÚnico population in municipalities of RJ for different income levels, per month

Now, we import the data about registration in CadÚnico and population by income groups (poverty/extreme poverty). This dataset is called "num_baixarenda_cadunico". A description of all variables used in this dataset is provided [here](https://dados.gov.br/dados/conjuntos-dados/cadastro-unico---familiaspessoas-por-faixas-de-renda-per-capita---mi-social).

We performed some data treatments to concatenate the CadÚnico DataFrames and generate a single dataset. Additionally, we created the "year" and "month" columns from the "anomes" column. Finally, we updated the dataset to its final version: we obtained the municipality names by matching the "cod_mun_ibge" column from the codigos_ibge_rj DataFrame with the "ibge" column of the num_baixarenda_cadunico DataFrame.

In [3]:
# Define the path to your zip file
zip_file_path = './Data/num_baixarenda_cadunico.zip'

# Initialize an empty list to store DataFrames
dfs = []

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    # Get a list of all CSV files in the zip archive
    csv_files = [name for name in zip_file.namelist() if name.endswith('.csv')]
    
    # Iterate through each CSV file
    for csv_file in csv_files:
        
        # Read the CSV file into a DataFrame and add the 'year' column
        df = pd.read_csv(zip_file.open(csv_file))
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
num_baixarenda_cadunico = pd.concat(dfs, ignore_index=True)

num_baixarenda_cadunico[['ibge','anomes']] = num_baixarenda_cadunico[['ibge','anomes']].astype(str)
num_baixarenda_cadunico['mes'] = num_baixarenda_cadunico['anomes'].str[4:6]
num_baixarenda_cadunico['ano'] = num_baixarenda_cadunico['anomes'].str[:4]

# Merge the DataFrames on the "Código Município Completo" and "ibge" columns
num_baixarenda_cadunico = num_baixarenda_cadunico.merge(codigos_ibge_rj, left_on='ibge', right_on='cod_mun_ibge', how='right')

# Drop the unnecessary columns if needed
num_baixarenda_cadunico.drop(['ibge'], axis=1, inplace=True)

# Print only some (useful) columns
num_baixarenda_cadunico = num_baixarenda_cadunico[['cod_mun_completo','cod_mun_ibge','nome_mun','cadunico_tot_pes','cadunico_tot_pes_rpc_ate_meio_sm','cadunico_tot_pes_pob','cadunico_tot_pes_ext_pob','cadunico_tot_pes_pob_e_ext_pob','cadunico_tot_fam','cadunico_tot_fam_rpc_ate_meio_sm','cadunico_tot_fam_pob','cadunico_tot_fam_ext_pob','cadunico_tot_fam_pob_e_ext_pob','mes','ano']]

num_baixarenda_cadunico.to_csv('./Data/Processed_Data/num_baixarenda_cadunico_rj.csv', index=False)

# Generation of dataset on the frequency of consumption of three meals in the municipalities of RJ, per year

In this case, we consider only the data related to the adult populations in these regions. A more comprehensive analysis could be performed by downloading data related to other populations (elderly, adolescents, etc.) if necessary. Simply download the corresponding datasets from the SISVAN website, concatenate them, and sum the ages corresponding to the same region/year. 

In [4]:
import pandas as pd

# Load the Excel file
xls = pd.ExcelFile('./Data/CONS_3REFEICOES_ADULTOS.xlsx')

# Initialize an empty list to store DataFrames
dfs = []

# Define the columns to be treated as strings
string_columns = ['cod_uf','cod_mun_ibge']

# Iterate through each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Read the sheet into a DataFrame, skipping the first row and renaming columns
    df = pd.read_excel(xls,
                       sheet_name,
                       skiprows=1,
                       names=['regiao', 'cod_uf', 'nome_uf', 'cod_mun_ibge', 'nome_mun', 'qtd_pessoas_tresref', 'porc_pessoas_tresref', 'total_adultos_acompanhados'],
                       dtype={col: str for col in string_columns})

    # Add a new column 'ano' with the sheet_name
    df['ano'] = sheet_name

    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames in the list into one unified DataFrame
cons_ref = pd.concat(dfs, ignore_index=True)

# Já existe uma coluna com o nome dos municípios, mas usarei como referência o cod_ibge para que a escrita seja a mesma do outro DataFrame. Por isso, vou editar a coluna nome_mun.

# Merge the DataFrames on the "Código Município Completo" and "ibge" columns
cons_ref = cons_ref.merge(codigos_ibge_rj, left_on='cod_mun_ibge', right_on='cod_mun_ibge', how='right')

# Drop the unnecessary columns if needed
cons_ref['nome_mun'] = cons_ref['nome_mun_y']
cons_ref.drop(['nome_mun_x','nome_mun_y'], axis=1, inplace=True)

cons_ref = cons_ref[['regiao','cod_uf','nome_uf','cod_mun_completo','cod_mun_ibge','nome_mun','qtd_pessoas_tresref','porc_pessoas_tresref', 'total_adultos_acompanhados', 'ano']]
cons_ref.to_csv('./Data/Processed_Data/cons_ref_rj.csv', index=False)

P.S. From this dataset, we can notice that several cities didn't register information about the consumption of meals of their citizens. This can be drawn from the fact that, for each year considered, we have less rows than actual cities (92) in the state. For instance, let's plot this table only for the year 2021.

In [5]:
cons_ref_2021 = cons_ref[cons_ref.ano=='2021'][['regiao','cod_uf','nome_uf','cod_mun_completo','cod_mun_ibge','nome_mun','qtd_pessoas_tresref','porc_pessoas_tresref', 'total_adultos_acompanhados', 'ano']]

print('In the year of 2021, only', len(cons_ref[cons_ref.ano=='2021']), 'cities have registered information in this dataset.')
cons_ref_2021


In the year of 2021, only 51 cities have registered information in this dataset.


,regiao,cod_uf,nome_uf,cod_mun_completo,cod_mun_ibge,nome_mun,qtd_pessoas_tresref,porc_pessoas_tresref,total_adultos_acompanhados,ano
6,SUDESTE,33,RJ,3300100,330010,Angra dos Reis,14.0,0.069307,202.0,2021
7,SUDESTE,33,RJ,3300159,330015,Aperibé,1.0,0.004274,234.0,2021
12,SUDESTE,33,RJ,3300209,330020,Araruama,21.0,0.117318,179.0,2021
19,SUDESTE,33,RJ,3300233,330023,Armação dos Búzios,0.0,0.000000,16.0,2021
21,SUDESTE,33,RJ,3300258,330025,Arraial do Cabo,0.0,0.000000,219.0,2021
26,SUDESTE,33,RJ,3300407,330040,Barra Mansa,548.0,0.109600,5000.0,2021
38,SUDESTE,33,RJ,3300704,330070,Cabo Frio,149.0,0.665179,224.0,2021
45,SUDESTE,33,RJ,3301009,330100,Campos dos Goytacazes,0.0,0.000000,24.0,2021
57,SUDESTE,33,RJ,3301207,330120,Carmo,92.0,0.730159,126.0,2021
59,SUDESTE,33,RJ,3300951,330095,Comendador Levy Gasparian,61.0,0.835616,73.0,2021


# Generation of dataset on BMI in municipalities of RJ, per year

Once again, we are considering only the data related to the adult populations in these regions. A more comprehensive analysis could be performed by downloading data related to other populations (elderly, adolescents, etc.). Similar to the previous case, databases for other age groups are available on the SISVAN platform.

In [6]:
# Define the path to your zip file
zip_file_path = './Data/IMC_RJ_DADOS.zip'

# Initialize an empty list to store DataFrames
dfs = []

string_columns = ['Código UF', 'Código IBGE']

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    # Get a list of all CSV files in the zip archive
    xlsx_files = [name for name in zip_file.namelist() if name.endswith('.xlsx')]
    
    # Iterate through each CSV file
    for xlsx_file in xlsx_files:
        # Extract the year from the filename
        year = xlsx_file.split('_')[2].split('.')[0]  # Assumes the filename structure is consistent
        
        # Read the CSV file into a DataFrame and add the 'year' column
        df = pd.read_excel(zip_file.open(xlsx_file),
                          dtype={col: str for col in string_columns})
        df['ano'] = int(year)
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
imc = pd.concat(dfs, ignore_index=True)
imc = imc.replace('-', 0.0)
imc.rename(columns={'Obesidade Grau I Porcentagem': 'Obesidade Grau I Quantidade'}, inplace=True) # Corrigindo nome errado de coluna

float_columns = ['Baixo Peso %', 'Adequado %', 'Sobrepeso %', 'Obesidade Grau I %', 'Obesidade Grau II %', 'Obesidade Grau III %']
for column in float_columns:
    imc[column] = imc[column].str.replace('%', '').astype(float)

# Já existe uma coluna com o nome dos municípios, mas usarei como referência o cod_ibge para que a escrita seja a mesma do outro DataFrame. Por isso, vou editar a coluna nome_mun.

# Merge the DataFrames on the "Código Município Completo" and "ibge" columns
imc = imc.merge(codigos_ibge_rj, left_on='Código IBGE', right_on='cod_mun_ibge', how='right')

imc.drop(['Município','Código IBGE'], axis=1, inplace=True)

imc = imc[['Região','Código UF','UF','cod_mun_completo','cod_mun_ibge','nome_mun',
          'Baixo Peso Quantidade', 'Baixo Peso %', 'Adequado Quantidade','Adequado %',
          'Sobrepeso Quantidade','Sobrepeso %', 'Obesidade Grau I Quantidade', 'Obesidade Grau I %',
          'Obesidade Grau II Quantidade', 'Obesidade Grau II %', 'Obesidade Grau III Quantidade',
          'Obesidade Grau III %', 'Total', 'ano']]

imc.to_csv('./Data/Processed_Data/imc_rj.csv', index=False)



# Generation of final datasets

In the final datasets generated (muns_demand_time), the goal is displaying monthly information about:

- City IDs
- City names
- Demand (estimated by the number of people classified in the "extreme poverty" threshold and registered in CadÚnico)
- Month
- Year
- City geometry (polygons for each city) 

In [7]:
geodata_rj = pd.read_csv("./Data/geodata_rj.csv")

# Add a column "demand" to the dataframe geodata_rj using "cadunico_tot_pes_ext_pob"

muns_demand_time = pd.read_csv("./Data/Processed_Data/num_baixarenda_cadunico_rj.csv")

geodata_rj['CD_MUN'] = geodata_rj['CD_MUN'].astype(str)
muns_demand_time['cod_mun_completo'] = muns_demand_time['cod_mun_completo'].astype(str)

# Merge DataFrames on common column
merged_df = pd.merge(muns_demand_time,
                     geodata_rj,
                     left_on='cod_mun_completo',
                     right_on='CD_MUN',
                     how='left')

muns_demand_time['geometry'] = merged_df['geometry']

# muns_demand_time.rename(columns={'cadunico_tot_pes_ext_pob':'demand'}, inplace=True)
# muns_demand_time.to_csv('./Data/Processed_Data/muns_demand_times.csv')

muns_demand_time.to_csv('./Data/Processed_Data/muns_demand_time.csv', index=False)

# As the file muns_demand_time is very big and may be hard to load in some machines, alternatively we generate the same dataset without the geometry column (which is responsible for the big weight)
muns_demand_time_no_geometry = muns_demand_time.drop(columns=['geometry'])
muns_demand_time_no_geometry.to_csv('./Data/Processed_Data/muns_demand_time_no_geometry.csv')



# Generating distance matrix, using Haversine distance

Generating a distance matrix using the Haversine distance is a crucial step in geographic and spatial analysis. The Haversine formula allows us to calculate the great-circle distance between two points on the Earth's surface, which is essential for various applications such as location-based services, routing, and optimization. By applying this formula systematically to a set of geographic coordinates, we can create a distance matrix that provides the pairwise distances between multiple locations, which will be used for our project. In our case, the result of the distance matrix will be provided in km.

In [22]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform

# Function to calculate Haversine distance in kilometers
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Calculate the distance
    distance = R * c
    
    return distance

# Assuming 'geodata_rj' contains 'centroid_x' and 'centroid_y' columns
coordinates = geodata_rj[['centroid_x', 'centroid_y']].values

# Calculate pairwise Haversine distances in kilometers
distance_matrix = squareform(pdist(coordinates, lambda u, v: haversine(u[0], u[1], v[0], v[1])))

# Convert the distance matrix to a DataFrame
distance_matrix_df = pd.DataFrame(distance_matrix, index=geodata_rj.index, columns=geodata_rj.index)

# Save the distance matrix to a CSV file (optional)
distance_matrix_df.to_csv('./Data/dist_matrix_muns.csv', index=False)

# Display the distance matrix
distance_matrix_df

,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
0,0.000000,269.270388,229.814164,149.615791,267.772600,243.876876,66.295114,42.423270,110.391632,229.278649,...,218.297048,200.709143,181.700065,173.183407,254.043509,154.351012,81.490139,328.013113,100.539794,49.196163
1,269.270388,0.000000,92.523311,119.654886,95.066585,104.961680,207.488254,238.609493,163.777537,52.201081,...,81.428710,69.721932,113.642252,98.433495,39.425833,115.359569,197.329907,71.341910,169.633888,227.713931
2,229.814164,92.523311,0.000000,101.461145,38.215995,20.336878,182.121653,211.726825,120.630517,46.499425,...,20.431619,67.310664,48.150155,73.967340,53.744102,104.659212,179.052887,162.137962,145.602647,200.857789
3,149.615791,119.654886,101.461145,0.000000,137.645230,120.881875,89.330499,120.595100,49.061161,83.424804,...,83.282339,51.918178,62.624379,28.327230,107.825960,9.043165,81.838331,180.980956,50.818309,109.519308
4,267.772600,95.066585,38.215995,137.645230,0.000000,26.431219,220.179232,249.904289,158.835488,66.794880,...,54.363598,96.657449,86.277915,109.506166,58.059464,139.862563,216.700187,156.459115,183.350363,239.016024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,154.351012,115.359569,104.659212,9.043165,139.862563,124.475580,92.293734,123.524782,57.135475,82.558243,...,85.699252,49.831827,68.665857,30.693292,106.436444,0.000000,83.350592,174.760220,54.276136,112.543979
88,81.490139,197.329907,179.052887,81.838331,216.700187,196.932341,16.535960,42.709554,66.582026,165.235892,...,162.845998,133.091287,133.464523,109.624559,189.507603,83.350592,0.000000,250.591169,33.528579,32.926601
89,328.013113,71.341910,162.137962,180.980956,156.459115,171.897734,263.165184,293.206042,228.459319,123.516802,...,152.598614,136.369833,184.601922,164.630811,108.422294,174.760220,250.591169,0.000000,227.538790,282.861408
90,100.539794,169.633888,145.602647,50.818309,183.350363,163.405793,38.549052,69.789119,36.726932,133.832397,...,129.698344,102.735838,99.969376,77.513588,158.413686,54.276136,33.528579,227.538790,0.000000,58.704854
